In [ ]:
#| echo: false
#| warning: false
from difflib import SequenceMatcher
import pandas as pd
import numpy as np
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
from scipy.spatial.distance import squareform
from sklearn.metrics import silhouette_score
import re
# from collection import Counter

The goal is to cluster together groups of similar ICD codes to aggregate any administrative statistics into intelligible categories.


In [ ]:
icd= pd.read_excel("C:\\Users\\macde\\OneDrive\\Desktop\\Projects\\Section111ValidICD10-Jan2023-DupFixed.xlsx")
icd['prefix'] = icd['CODE'].str.slice(0,1)
icd = icd[icd['prefix']=='F']
icd = icd.reset_index()
icd['index'] = icd.index

Here we're just looking at F-category (mental/psych) codes. This saves us a little computing power and accuracy because the letters are intended to capture some similarity among codes.


In [ ]:
stopwords = ['the', 'of', 'and', 'with', 'without', 'other', 'to', 'due', 'a', 'specified', 'unspecified']

def str_cleaner(str_in):
  str_in = str_in.lower()
  str_list = str_in.split()
  
  resultwords = [word for word in str_list if word.lower() not in stopwords]
  result = ' '.join(resultwords)
  
  str_out = re.sub(r'\W+', '', result)
  
  return str_out

icd['str_out'] = icd["SHORT DESCRIPTION (VALID ICD-10 FY2023)"].apply(str_cleaner)
icd

asdf

In [ ]:
N = len(icd)
dist_mat = np.zeros([N,N])

for i in range(N):
  for j in range(i):
    wi = icd.loc[i,'str_out']
    wj = icd.loc[j,'str_out']
    if i==j:
      simm = 1
    else:
      simm = SequenceMatcher(None,wi, wj).ratio()
    dist_mat[i,j] = simm
    dist_mat[j,i] = simm
    
dist_mat = 1-dist_mat

In [ ]:
plt.figure(figsize=(7,7))
plt.imshow(dist_mat,cmap='hot')
plt.show()


A range of 859 ICD-10 codes is now grouped into 282 families.

Choices that impact clustering:

-   Tokenizing

-   Distance function

-   agg vs div vs other clustering

-   AGNES method

-   number of groups

## To-Do:

Investigate a loss function to search parameters and reduce analyst intervention.